This notebook is to test my `torchnetjson` (`JSONNet`) framework to generate the basic RCNN plus a readout, just as in `maskcnn_polished` and `maskcnn_polished_with_local_pcn`. I did this so that to make sure I can generate the `builder` files for this network easily. `builder` files are required for network training in my thesis codebase.

In [1]:
import numpy as np
from torch import tensor, no_grad, Tensor

from thesis_v2.blocks.rcnn_basic_kriegeskorte import nn_modules
from thesis_v2.blocks import load_modules
from thesis_v2.blocks_json.utils import new_map_size
from thesis_v2.blocks_json import maskcnn, pooling

In [2]:
np.random.seed(0)
random_img = np.random.random_sample((10, 50, 50, 1))*2 - 1
random_img = random_img.astype(np.float32)
random_img = random_img.transpose((0,3,1,2))

In [3]:
# a = nn_modules.BLConvLayerStack(
#     n_timesteps=8,
#     channel_list=[3,96,128,192,256,512,1024,2048],
#     ksize_list=[7,5,3,3,3,3,1],
#     # matching tf's bn config.
#     bn_eps=0.001,
# )

from torch import tensor
from torchnetjson.builder import build_net

load_modules([
    'rcnn_kriegeskorte.blstack',
    'rcnn_kriegeskorte.accumulator',
    'maskcnn.factoredfc',
])

def build_test_net(out, *,
                   input_size=(50,50),
                   have_readout=False,
                  ):
    input_size_dict = {
        'map_size': input_size,
    }
    module_dict = {
        'stack': {
            'name':'rcnn_kriegeskorte.blstack',
            'params': {
                'n_timesteps': 8,
                'channel_list': [1,32,16],
                'ksize_list': [7,3],
                # no pooling.
                'pool_ksize': 1,
            },
            'init': {
                'strategy': 'normal',
                'parameters': {'std': 0.5}
            }
        },
        'accumulator_cummean': {
            'name': 'rcnn_kriegeskorte.accumulator',
            'params': {'mode': 'cummean'},
            'init': None,
        },
        **pooling.pool2d(
            name='pooling',
             pooling_type='max',
             kernel_size=2,
             input_size=input_size_dict['map_size'],
             state_dict=input_size_dict,
             ceil_mode=False,
             map_size_strict=True
        ),
        **maskcnn.factoredfc(
            name='fc',
            map_size=input_size_dict['map_size'],
            in_channels=16,
            out_features=5,
            bias=True,
            weight_spatial_constraint=None,
            weight_feature_constraint=None,
        )
    }
    
    # because this whole network uses `same` padding, in TF jargon and no pooling.
    map_size = input_size
    
    intermediate_name = 'inter_unused'

    op_list = [
        {'name': 'module',
         'args': ['stack'],
         'kwargs': {},
         'in': 'input0',
         'out': 'out_raw',
        },
        {'name': 'module',
         'args': ['accumulator_cummean'],
         'kwargs': {
             'unpack': False,
         },
         'in': 'out_raw',
         'out': 'out_cummean',
        },
        # then apply pooling to `out_cummean`, using `module_repeat` op.
        {'name': 'module_repeat',
         'args': ['pooling'],
         'kwargs': {},
         'in': 'out_cummean',
         'out': intermediate_name,
        },
        {'name': 'module_repeat',
         'args': ['fc'],
         'kwargs': {},
         'in': intermediate_name,
         'out': 'fc_out',
        },
        # finally, combine all timesteps' data into one chunk.
        {
            'name': 'stack',
            'args': [],
            'kwargs': {'dim': 0},
            'in': 'fc_out',
            'out': 'combined',
        }
    ]

    param_dict = {
        'module_dict': module_dict,
        'op_list': op_list,
        # change out to fit your needs.
        'out': out,
    }

    return build_net(param_dict)


a = build_test_net('combined')

In [4]:
def check_build_net_params(net_to_check):
    with no_grad():
        for x, y in net_to_check.named_parameters():
            print(x, y.size(), y.mean(), y.std())
        
# looks good. for conv kernels, mean should be 0, std should be 0.5,
# for bn parameters, weight should all be 1, bias should all be 0.
# for fc layer parameters, should have zero mean and std of 0.01 (the default value)
check_build_net_params(a)

moduledict.fc.weight_spatial torch.Size([5, 25, 25]) tensor(-0.0003) tensor(0.0101)
moduledict.fc.weight_feature torch.Size([5, 16]) tensor(0.0005) tensor(0.0087)
moduledict.fc.bias torch.Size([5]) tensor(0.) tensor(0.)
moduledict.stack.layer_list.0.b_conv.weight torch.Size([32, 1, 7, 7]) tensor(0.0181) tensor(0.5002)
moduledict.stack.layer_list.0.l_conv.weight torch.Size([32, 32, 7, 7]) tensor(0.0004) tensor(0.4988)
moduledict.stack.layer_list.1.b_conv.weight torch.Size([16, 32, 3, 3]) tensor(-0.0094) tensor(0.4957)
moduledict.stack.layer_list.1.l_conv.weight torch.Size([16, 16, 3, 3]) tensor(0.0148) tensor(0.4911)
moduledict.stack.bn_layer_list.0.weight torch.Size([32]) tensor(1.) tensor(0.)
moduledict.stack.bn_layer_list.0.bias torch.Size([32]) tensor(0.) tensor(0.)
moduledict.stack.bn_layer_list.1.weight torch.Size([16]) tensor(1.) tensor(0.)
moduledict.stack.bn_layer_list.1.bias torch.Size([16]) tensor(0.) tensor(0.)
moduledict.stack.bn_layer_list.2.weight torch.Size([32]) tensor(

In [5]:
# let's put some test data.
a.cuda().eval()

with no_grad():
    out_this = a(tensor(random_img).cuda())
    print(type(out_this))
#     print(Tensor)
    
    if isinstance(out_this, Tensor):
        print(out_this.type())
    for x in out_this:
        print(x.size())
        
# also tried a = build_test_net('inter_unused'), get eight torch.Size([10, 16, 25, 25]), as expected.


<class 'torch.Tensor'>
torch.cuda.FloatTensor
torch.Size([10, 5])
torch.Size([10, 5])
torch.Size([10, 5])
torch.Size([10, 5])
torch.Size([10, 5])
torch.Size([10, 5])
torch.Size([10, 5])
torch.Size([10, 5])
